<a href="https://colab.research.google.com/github/deegrayce/Anagram-game/blob/main/Chicago_crime_analysis_circle_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chicago Crime Rate Analysis**
**Abbreviations**

IUCR - Illinois Uniform Crime Reporting

ID - Case identifier

District - a region often larger than a city but smaller than a state.

Community area - An area found within a city or district, it encompasses multiple wards.

Ward - wards are smaller subdivisions  found in a community or local government area.


## **Data Cleaning**

In [148]:
import pandas as pd

In [149]:
filepath = "/content/chicago3.csv"
df = pd.read_csv(filepath)
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,201000,3653721,HK742237,10/25/2004 12:00:00 AM,008XX N HOMAN AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,...,27.0,23.0,26,1153552.0,1905563.0,2004,02/28/2018 03:56:25 PM,41.896706,-87.711477,"(41.896705626, -87.711476738)"
1,201001,3105528,HJ837008,12/24/2003 08:30:00 PM,061XX N AVERS AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,...,39.0,13.0,05,1149579.0,1940599.0,2003,02/28/2018 03:56:25 PM,41.992925,-87.725156,"(41.99292531, -87.725155969)"
2,201002,3761615,HL132238,01/18/2005 11:11:31 PM,045XX N MILWAUKEE AVE,0496,BATTERY,AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,RESIDENCE,False,...,45.0,15.0,04B,1140866.0,1930028.0,2005,02/28/2018 03:56:25 PM,41.964083,-87.757467,"(41.96408297, -87.757467311)"
3,201003,3613882,HK707251,10/24/2004 03:05:00 PM,012XX W BELMONT AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,OTHER,True,...,32.0,6.0,14,1167442.0,1921381.0,2004,02/10/2018 03:50:01 PM,41.939824,-87.660005,"(41.939823537, -87.660005051)"
4,201004,1504341,G247242,04/30/2001 05:52:55 PM,027XX W 23 PL,0460,BATTERY,SIMPLE,STREET,False,...,NaN,NaN,08B,1158699.0,1888329.0,2001,08/17/2015 03:03:40 PM,41.849310,-87.693045,"(41.849309923, -87.693044786)"


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1000 non-null   int64  
 1   ID                    1000 non-null   int64  
 2   Case Number           1000 non-null   object 
 3   Date                  1000 non-null   object 
 4   Block                 1000 non-null   object 
 5   IUCR                  1000 non-null   object 
 6   Primary Type          1000 non-null   object 
 7   Description           1000 non-null   object 
 8   Location Description  1000 non-null   object 
 9   Arrest                1000 non-null   bool   
 10  Domestic              1000 non-null   bool   
 11  Beat                  1000 non-null   int64  
 12  District              1000 non-null   float64
 13  Ward                  928 non-null    float64
 14  Community Area        927 non-null    float64
 15  FBI Code              

In [151]:
#dropping redundant columns but first we check to see what the redundant columns contain
df[["Primary Type", "Description", "Location Description"]]

,Primary Type,Description,Location Description
0,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE
1,BURGLARY,FORCIBLE ENTRY,RESIDENCE
2,BATTERY,AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,RESIDENCE
3,CRIMINAL DAMAGE,TO PROPERTY,OTHER
4,BATTERY,SIMPLE,STREET
...,...,...,...
995,THEFT,OVER $500,GAS STATION
996,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,SMALL RETAIL STORE
997,ROBBERY,ARMED: HANDGUN,TAXICAB
998,BURGLARY,FORCIBLE ENTRY,PARK PROPERTY


In [152]:
#indexing to check if the columns are redundant. They are giving different information so we will be keeping all 3
#another reason for keeping them is for area drilling purposes, to get deeper insights, a district is the largest, followed by community area then Ward.
df[["District", "Community Area", "Ward", "Block"]]

,District,Community Area,Ward,Block
0,11.0,23.0,27.0,008XX N HOMAN AVE
1,17.0,13.0,39.0,061XX N AVERS AVE
2,16.0,15.0,45.0,045XX N MILWAUKEE AVE
3,19.0,6.0,32.0,012XX W BELMONT AVE
4,10.0,NaN,NaN,027XX W 23 PL
...,...,...,...,...
995,15.0,25.0,24.0,003XX S CICERO AVE
996,18.0,8.0,42.0,0000X E OAK ST
997,25.0,22.0,30.0,023XX N HAMLIN AVE
998,3.0,NaN,NaN,064XX S RICHARDS DR


In [153]:
#checking to see if there is a pattern to the FBI Code

df["FBI Code"].unique()

array(['26', '05', '04B', '14', '08B', '22', '06', '08A', '11', '04A',
       '03', '10', '16', '18', '07', '17', '15', '24', '02', '20', '09'],
      dtype=object)

In [154]:
#dropping redundant columns
df.drop(["Unnamed: 0", "Case Number", "X Coordinate", "Y Coordinate", "IUCR"], axis =1, inplace = True)
df.head()

,ID,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Updated On,Latitude,Longitude,Location
0,3653721,10/25/2004 12:00:00 AM,008XX N HOMAN AVE,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,1121,11.0,27.0,23.0,26,2004,02/28/2018 03:56:25 PM,41.896706,-87.711477,"(41.896705626, -87.711476738)"
1,3105528,12/24/2003 08:30:00 PM,061XX N AVERS AVE,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,1711,17.0,39.0,13.0,05,2003,02/28/2018 03:56:25 PM,41.992925,-87.725156,"(41.99292531, -87.725155969)"
2,3761615,01/18/2005 11:11:31 PM,045XX N MILWAUKEE AVE,BATTERY,AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,RESIDENCE,False,True,1623,16.0,45.0,15.0,04B,2005,02/28/2018 03:56:25 PM,41.964083,-87.757467,"(41.96408297, -87.757467311)"
3,3613882,10/24/2004 03:05:00 PM,012XX W BELMONT AVE,CRIMINAL DAMAGE,TO PROPERTY,OTHER,True,False,1932,19.0,32.0,6.0,14,2004,02/10/2018 03:50:01 PM,41.939824,-87.660005,"(41.939823537, -87.660005051)"
4,1504341,04/30/2001 05:52:55 PM,027XX W 23 PL,BATTERY,SIMPLE,STREET,False,False,1034,10.0,NaN,NaN,08B,2001,08/17/2015 03:03:40 PM,41.849310,-87.693045,"(41.849309923, -87.693044786)"


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1000 non-null   int64  
 1   Date                  1000 non-null   object 
 2   Block                 1000 non-null   object 
 3   Primary Type          1000 non-null   object 
 4   Description           1000 non-null   object 
 5   Location Description  1000 non-null   object 
 6   Arrest                1000 non-null   bool   
 7   Domestic              1000 non-null   bool   
 8   Beat                  1000 non-null   int64  
 9   District              1000 non-null   float64
 10  Ward                  928 non-null    float64
 11  Community Area        927 non-null    float64
 12  FBI Code              1000 non-null   object 
 13  Year                  1000 non-null   int64  
 14  Updated On            1000 non-null   object 
 15  Latitude              

In [156]:
#checking the total null values
df.isnull().sum()

ID                       0
Date                     0
Block                    0
Primary Type             0
Description              0
Location Description     0
Arrest                   0
Domestic                 0
Beat                     0
District                 0
Ward                    72
Community Area          73
FBI Code                 0
Year                     0
Updated On               0
Latitude                12
Longitude               12
Location                12
dtype: int64

In [157]:
#indexing the datafarme and filtering out the columns on location to see if we can make some connections and fill in the missing rowas
#we got all the location data containing null values alone
location_data_null = df[["District", "Community Area", "Ward"]][df[["District", "Community Area", "Ward"]].isnull().any(axis = 1)]
location_data_null

,District,Community Area,Ward
4,10.0,NaN,NaN
11,12.0,NaN,NaN
16,14.0,NaN,NaN
17,5.0,NaN,NaN
37,11.0,NaN,NaN
...,...,...,...
908,10.0,NaN,NaN
930,6.0,NaN,NaN
941,17.0,NaN,NaN
994,15.0,NaN,NaN


In [158]:
#There are 21 districts with null values in community area and ward, we are going to back fill them by district to ensure precision
location_data_null['District'].unique()

array([10., 12., 14.,  5., 11.,  6., 15.,  3.,  9., 17., 25.,  2.,  8.,
        1.,  4., 20.,  7., 22., 18., 24., 19.])

In [159]:
#indexing for the data's  general location
location_data = df[["District", "Community Area", "Ward"]]
location_data

,District,Community Area,Ward
0,11.0,23.0,27.0
1,17.0,13.0,39.0
2,16.0,15.0,45.0
3,19.0,6.0,32.0
4,10.0,NaN,NaN
...,...,...,...
995,15.0,25.0,24.0
996,18.0,8.0,42.0
997,25.0,22.0,30.0
998,3.0,NaN,NaN


In [160]:
#using backward fill to fill the missing values
#forward fill would not work here because we are filling community area and ward by district and not just randomly to ensure as much accuracy as we can get
def backward_fill_by_district(location_data):
  #extracting unique district numbers so we can fill based on district numbers
    districts = location_data['District'].unique()
    for district in districts:
        mask = location_data['District'] == district
        # initiating a loop that iterates over each unique district number stored in the `districts` variable
        location_data.loc[mask] = location_data.loc[mask].fillna(method='bfill', axis=0)
    return location_data

# Applying the function to your dataset
location_data_filled = backward_fill_by_district(location_data)


<ipython-input-160-ff7547c51ede>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_data.loc[mask] = location_data.loc[mask].fillna(method='bfill', axis=0)


In [161]:
#equating it so that the filled data will reflect on our dataset df
df.loc[:, ['District', 'Community Area', 'Ward']] = location_data_filled.loc[:, ['District', 'Community Area', 'Ward']]
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1000 non-null   int64  
 1   Date                  1000 non-null   object 
 2   Block                 1000 non-null   object 
 3   Primary Type          1000 non-null   object 
 4   Description           1000 non-null   object 
 5   Location Description  1000 non-null   object 
 6   Arrest                1000 non-null   bool   
 7   Domestic              1000 non-null   bool   
 8   Beat                  1000 non-null   int64  
 9   District              1000 non-null   float64
 10  Ward                  999 non-null    float64
 11  Community Area        999 non-null    float64
 12  FBI Code              1000 non-null   object 
 13  Year                  1000 non-null   int64  
 14  Updated On            1000 non-null   object 
 15  Latitude              

In [162]:
#backward filling latitude and location
df["Latitude"].bfill(axis = 0, inplace = True)
df["Latitude"].isnull().sum()


0

In [163]:
#backward filling longitude
df["Longitude"].bfill(axis = 0, inplace = True)
df["Longitude"].isnull().sum()

0

In [164]:
#backward filling location
df["Location"].bfill(axis = 0, inplace = True)
df["Location"].isnull().sum()

0

In [165]:
#ensuring we have no missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1000 non-null   int64  
 1   Date                  1000 non-null   object 
 2   Block                 1000 non-null   object 
 3   Primary Type          1000 non-null   object 
 4   Description           1000 non-null   object 
 5   Location Description  1000 non-null   object 
 6   Arrest                1000 non-null   bool   
 7   Domestic              1000 non-null   bool   
 8   Beat                  1000 non-null   int64  
 9   District              1000 non-null   float64
 10  Ward                  999 non-null    float64
 11  Community Area        999 non-null    float64
 12  FBI Code              1000 non-null   object 
 13  Year                  1000 non-null   int64  
 14  Updated On            1000 non-null   object 
 15  Latitude              

In [166]:
#singling out the null value that was not filled
df[["District", "Community Area", "Ward"]][df[["District", "Community Area", "Ward"]].isnull().any(axis = 1)]


,District,Community Area,Ward
998,3.0,NaN,NaN


In [167]:
df[df["District"] == 3.0]

,ID,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Updated On,Latitude,Longitude,Location
10,7406204,03/14/2010 03:30:00 PM,061XX S VERNON AVE,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,313,3.0,20.0,42.0,14,2010,02/10/2018 03:50:01 PM,41.782991,-87.614478,"(41.782990756, -87.614478366)"
12,5601683,06/16/2007 08:40:00 PM,070XX S SOUTH CHICAGO AVE,OTHER OFFENSE,FALSE/STOLEN/ALTERED TRP,STREET,False,False,322,3.0,6.0,69.0,26,2007,02/28/2018 03:56:25 PM,41.767280,-87.607553,"(41.767280058, -87.607553429)"
14,9410476,12/01/2013 05:00:00 AM,006XX E 61ST ST,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,SIDEWALK,False,False,313,3.0,20.0,42.0,03,2013,02/10/2018 03:50:01 PM,41.783973,-87.610581,"(41.783973454, -87.610580799)"
22,4582467,01/28/2006 03:21:44 PM,067XX S CHAPPEL AVE,BATTERY,SIMPLE,RESIDENCE,True,False,331,3.0,5.0,43.0,08B,2006,02/28/2018 03:56:25 PM,41.773007,-87.575261,"(41.773007423, -87.575261338)"
46,11033474,06/02/2017 12:00:00 PM,071XX S CYRIL AVE,THEFT,$500 AND UNDER,APARTMENT,False,False,333,3.0,5.0,43.0,06,2017,02/10/2018 03:50:01 PM,41.764901,-87.576944,"(41.764900898, -87.576943938)"
89,1365481,02/04/2001 06:16:52 PM,073XX S DORCHESTER AV,BATTERY,AGGRAVATED: OTHER DANG WEAPON,APARTMENT,False,True,324,3.0,6.0,69.0,04B,2001,08/17/2015 03:03:40 PM,41.760638,-87.590706,"(41.760638224, -87.59070575)"
102,2991651,10/11/2003 03:30:00 PM,074XX S MARYLAND AVE,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,False,False,323,3.0,6.0,69.0,26,2003,02/28/2018 03:56:25 PM,41.759967,-87.604180,"(41.759967099, -87.60417953)"
136,12238151,12/06/2020 01:02:00 AM,062XX S MICHIGAN AVE,BATTERY,SIMPLE,STREET,False,False,311,3.0,20.0,40.0,08B,2020,12/17/2020 03:44:58 PM,41.781405,-87.622009,"(41.781404777, -87.622008635)"
153,8293901,10/05/2011 01:45:00 AM,009XX E 75TH ST,BATTERY,SIMPLE,SIDEWALK,True,False,323,3.0,8.0,69.0,08B,2011,02/04/2016 06:33:39 AM,41.758626,-87.601336,"(41.758626266, -87.601335891)"
260,7825136,11/28/2010 05:58:00 PM,005XX E 62ND ST,ROBBERY,STRONGARM - NO WEAPON,STREET,False,False,313,3.0,20.0,42.0,03,2010,02/10/2018 03:50:01 PM,41.782110,-87.612783,"(41.782110197, -87.612783284)"


In [168]:
#checking for duplicates
df.duplicated().sum()

0

In [169]:
#changing the data column to datetime format and extracting season
df['Date'] = pd.to_datetime(df['Date'])

#extracting month
from datetime import datetime

def get_month(date):
    #date_column = pd.to_datetime(date, format='%m/%d/%Y %I:%M:%S %p')

    month_names = df['Date'].dt.month_name()

    return month_names

df['Month'] = get_month(df['Date'])
#checking to seee that it has been added
df.columns

Index(['ID', 'Date', 'Block', 'Primary Type', 'Description',
       'Location Description', 'Arrest', 'Domestic', 'Beat', 'District',
       'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Month'],
      dtype='object')

In [172]:
# creating a column for seasons
df['Date'] = pd.to_datetime(df['Date'])

def get_seasons(date_column):

    # Define a mapping of month to season
    month_to_season = {
        1: 'Winter', 2: 'Winter', 3: 'Spring',
        4: 'Spring', 5: 'Spring', 6: 'Summer',
        7: 'Summer', 8: 'Summer', 9: 'Fall',
        10: 'Fall', 11: 'Fall', 12: 'Winter'
    }

    # Extract the month and map to season
    seasons_column = df['Date'].dt.month.map(month_to_season)


    return seasons_column
df['Seasons'] = get_seasons(df['Date'])
df.columns

Index(['ID', 'Date', 'Block', 'Primary Type', 'Description',
       'Location Description', 'Arrest', 'Domestic', 'Beat', 'District',
       'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Month', 'Seasons'],
      dtype='object')

#**Exploratoty Data Analysis**

In [173]:
df.describe()

,ID,Beat,District,Ward,Community Area,Year,Latitude,Longitude
count,1.000000e+03,1000.000000,1000.00000,999.000000,999.000000,1000.000000,1000.000000,1000.000000
mean,6.824392e+06,1163.073000,10.94900,22.334334,36.433433,2009.521000,41.843796,-87.669804
std,3.392723e+06,687.149871,6.77601,13.755754,20.858271,5.968684,0.085412,0.059871
min,1.322829e+06,111.000000,1.00000,1.000000,1.000000,2001.000000,41.644590,-87.905227
25%,3.701280e+06,613.000000,5.00000,9.000000,23.000000,2004.000000,41.770275,-87.712537
50%,6.702616e+06,1112.000000,10.00000,23.000000,32.000000,2009.000000,41.859861,-87.663935
75%,9.749364e+06,1712.250000,16.00000,33.000000,53.000000,2014.000000,41.904246,-87.626388
max,1.276971e+07,2535.000000,25.00000,50.000000,77.000000,2022.000000,42.020291,-87.524637
